In [ ]:
import requests
import json
import pandas as pd
import math

base_url = 'https://fantasy.premierleague.com/api/'


def get_transfers(league):
    # Get manager details
    general_info = requests.get(
        base_url + "leagues-h2h-matches/league/" + str(league) + "/").json()

    # For each manager id, get the the details
    # for id_ in manager_ids:
    manager_details_ = [{"manager_id": manager['entry_1_entry'], "player_name": manager['entry_1_player_name'],
                         "name": manager['entry_1_name']} for manager in general_info['results']]
    manager_details = list(
        {manager['manager_id']: manager for manager in manager_details_}.values())

    # For each manager, get their pick history
    manager_picks = []
    for manager in manager_details:
        # For leagues with teams, a virtual manager is include.
        # Check if a manager_id is 'None' and skip that manager
        if (manager['manager_id'] == None):
            continue

        picks = []
        for gw in range(1, 39):
            try:
                gw_picks = requests.get(
                    base_url + "entry/" + str(manager['manager_id']) + "/event/" + str(gw) + "/picks/").json()
            except requests.exceptions.JSONDecodeError as jsonerror:
                print("JSONEconding Error: ", jsonerror, ", ",
                      gw, '=========> ', manager['manager_id'])
            except requests.exceptions.HTTPError as errh:
                print("Http Error:", errh)
            except requests.exceptions.ConnectionError as errc:
                print("Error Connecting:", errc)
            except requests.exceptions.Timeout as errt:
                print("Timeout Error:", errt)
            except requests.exceptions.RequestException as err:
                print("OOps: Something Else", err)

        picks = []
        for gw in range(1, 39):
            gw_picks = requests.get(
                base_url + "entry/" + str(manager['manager_id']) + "/event/" + str(gw) + "/picks/").json()
            picks.append({gw: gw_picks})
        manager_picks.append({manager['manager_id']: picks})

    return {league: manager_picks}

In [ ]:
# Get league ids between 2123000 and 2123881
league_ids = []
for i in range(2125120, 2126441):  # 2123882, 2124764 ,  2124089
    general_info = requests.get(
        base_url + "leagues-h2h-matches/league/" + str(i) + "/").json()
    # Check if the response contains a results list and it is not empty
    if ('results' in general_info and general_info['results']):
        league_ids.append(i)
print('leagues collected')
# league_history = [get_transfers(league) for league in league_ids]
len(league_ids)

In [ ]:
for league in league_ids:
    print('==> Fetching league transfer details for ', league)
    leagues_transfer_details = get_transfers(league)

    print('<== Completed fetching league transfer details for ', league)
    # Append the manager league details to a json file
    data_file_contents = []
    with open("/content/drive/MyDrive/Colab Notebooks/h2h_league_picks_5.json") as data_file:
        data_file_contents = json.load(data_file)

    # # parsed_json = json.loads(file_contents)
    with open("/content/drive/MyDrive/Colab Notebooks/h2h_league_picks_5.json", mode='w',  encoding='utf-8') as f:
        data_file_contents.append(leagues_transfer_details)
        json.dump(data_file_contents, f)

    # Log some details to show progress
    print(league, ':::::::::::',  round(
        (league_ids.index(league)+1)/len(league_ids)*100, 1), "%", " done")

In [6]:
import json
import pandas as pd
data_file_contents = []
matches_data_file = []

with open("./content/h2h_league/h2h_league_picks_2.json", mode='r') as data_file:
    data_file_contents = json.load(data_file)

with open("./content/h2h_matches.json", mode='r') as data_file:
    matches_data_file = json.load(data_file)

In [36]:
# league_id, manager_id, gw, active_chip, automatic_subs, gw_points, total_points, picks: (), captain, vice_captain, bank, opponent, opponent_points, results,

managers = []


# league_keys = [list(league.keys())[0] for league in data_file_contents]
for league in data_file_contents:
    # for match in matches_data_file:
    #     print(int(list(league.keys())[0]), match)
    # # league_matches = list(list(filter(lambda x: int(list(x.keys())[0]) == int(list(league.keys())[0]), matches_data_file))[0].values())[0]

    for manager in league[list(league.keys())[0]]:
        for gw in manager[list(manager.keys())[0]]:

            active_chip = gw[list(gw.keys())[0]]['active_chip']
            automatic_subs = gw[list(gw.keys())[0]]['automatic_subs']
            gw_points = gw[list(gw.keys())[0]]['entry_history']['points']
            total_points = gw[list(gw.keys())[0]
                              ]['entry_history']['total_points']

            picks = tuple([player['element']
                           for player in gw[list(gw.keys())[0]]['picks']])
            captain = None
            vice_captain = None

            for player in gw[list(gw.keys())[0]]['picks']:
                if (player['is_captain']):
                    captain = player['element']
                if (player['is_vice_captain']):
                    vice_captain = player['element']

            bank = gw[list(gw.keys())[0]]['entry_history']['bank']
            manager_history = {"league_id": list(league.keys())[0], 'manager_id': list(manager.keys())[0], 'gw': list(gw.keys())[0],
                               'active_chip': active_chip, 'automatic_subs': automatic_subs, 'gw_points': gw_points, 'total_points': total_points, 'picks': picks, 'captain': captain, 'vice_captain': vice_captain, 'bank': bank}
            managers.append(manager_history)


managers

not in {'9393679': [{'1': {'detail': 'Not found.'}}, {'2': {'detail': 'Not found.'}}, {'3': {'detail': 'Not found.'}}, {'4': {'active_chip': 'bboost', 'automatic_subs': [], 'entry_history': {'event': 4, 'points': 97, 'total_points': 97, 'rank': 181463, 'rank_sort': 194227, 'overall_rank': 9218631, 'percentile_rank': None, 'bank': 0, 'value': 1001, 'event_transfers': 0, 'event_transfers_cost': 0, 'points_on_bench': 0}, 'picks': [{'element': 291, 'position': 1, 'multiplier': 1, 'is_captain': False, 'is_vice_captain': False}, {'element': 290, 'position': 2, 'multiplier': 1, 'is_captain': False, 'is_vice_captain': False}, {'element': 369, 'position': 3, 'multiplier': 1, 'is_captain': False, 'is_vice_captain': False}, {'element': 20, 'position': 4, 'multiplier': 1, 'is_captain': False, 'is_vice_captain': False}, {'element': 506, 'position': 5, 'multiplier': 1, 'is_captain': False, 'is_vice_captain': False}, {'element': 216, 'position': 6, 'multiplier': 1, 'is_captain': False, 'is_vice_capta

KeyError: 'automatic_subs'

In [31]:
managers_pd = pd.DataFrame(managers)

In [32]:
managers_pd.to_csv('./content/managers.csv')

In [73]:
import json_stream

data_path = "content/h2h_league_picks.json"

# data = json_stream.load(data_path)

df = pd.read_json(data_path, lines=True)

ValueError: No ':' found when decoding object value